### data load

In [ ]:
## C:\Users\DongJin\AppData\Local\Temp\cuda -> cuda 경로

import pandas as pd
import numpy as np
import os
import cv2


In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten



In [3]:

# dataset 구조 확인
driver_details = pd.read_csv('C:/Users/DongJin/Desktop/YS/DL/state-farm-distracted-driver-detection/driver_imgs_list.csv', na_values='na')
print(driver_details.head(5))

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [4]:
## Getting all the images

train_image = []
image_label = []


for i in range(10):
    print('now we are in the folder C',i)
    imgs = os.listdir("C:/Users/DongJin/Desktop/YS/DL/state-farm-distracted-driver-detection/imgs/train/c"+str(i))
    for j in range(len(imgs)):
    #for j in range(100):
        img_name = "C:/Users/DongJin/Desktop/YS/DL/state-farm-distracted-driver-detection/imgs/train/c"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        #img = color.rgb2gray(img)
        img = img[50:,120:-50]
        img = cv2.resize(img,(224,224))
        label = i
        driver = driver_details[driver_details['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])
        image_label.append(i)

now we are in the folder C 0
now we are in the folder C 1
now we are in the folder C 2
now we are in the folder C 3
now we are in the folder C 4
now we are in the folder C 5
now we are in the folder C 6
now we are in the folder C 7
now we are in the folder C 8
now we are in the folder C 9


In [5]:
## Randomly shuffling the images
## 이미지가 순서대로 c0부터 c9까지 정렬되어있기 떄문에, 섞어줌

import random
random.shuffle(train_image)

In [6]:
## getting list of driver names

D = []
for features,labels,drivers in train_image:
    D.append(drivers)

## Deduplicating drivers

deduped = []  # driver들

for i in D:
    if i not in deduped:
        deduped.append(i)
    

## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 4)
for i in driv_nums:
    driv_selected.append(deduped[i])

In [7]:
print(driv_selected, '\n', deduped, '\n', D)

['p075', 'p015', 'p024', 'p052'] 
 ['p052', 'p016', 'p047', 'p026', 'p051', 'p075', 'p039', 'p035', 'p045', 'p021', 'p014', 'p066', 'p064', 'p024', 'p050', 'p081', 'p012', 'p015', 'p022', 'p002', 'p041', 'p061', 'p056', 'p049', 'p072', 'p042'] 
 ['p052', 'p016', 'p047', 'p026', 'p051', 'p075', 'p039', 'p035', 'p045', 'p021', 'p014', 'p066', 'p064', 'p045', 'p024', 'p035', 'p050', 'p039', 'p052', 'p081', 'p012', 'p021', 'p052', 'p035', 'p014', 'p039', 'p015', 'p022', 'p075', 'p024', 'p035', 'p052', 'p002', 'p016', 'p039', 'p041', 'p075', 'p052', 'p061', 'p026', 'p066', 'p022', 'p021', 'p014', 'p075', 'p014', 'p081', 'p050', 'p066', 'p045', 'p024', 'p066', 'p066', 'p022', 'p022', 'p056', 'p081', 'p021', 'p061', 'p002', 'p024', 'p050', 'p014', 'p024', 'p022', 'p021', 'p012', 'p051', 'p026', 'p066', 'p081', 'p014', 'p024', 'p064', 'p016', 'p049', 'p024', 'p051', 'p047', 'p021', 'p056', 'p015', 'p066', 'p021', 'p014', 'p012', 'p041', 'p072', 'p035', 'p024', 'p061', 'p051', 'p021', 'p039', '

In [8]:
## Splitting the train and test

X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

for features,labels,drivers in train_image:
    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        D_train.append(drivers)
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))

18769 3655
18769 3655


In [9]:
## Converting images to nparray. Encoding the Y
import tensorflow as tf
from keras.utils import to_categorical

X_train = tf.image.resize(X_train, [244, 244]).numpy().astype('float32')/255.0
X_test = tf.image.resize(X_test, [244, 244]).numpy().astype('float32')/255.0


In [10]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


print (X_train.shape)

(18769, 244, 244, 3)


In [11]:
import numpy as np

# X_train과 X_test 데이터를 각각 .npy 파일로 저장
np.save('X_train_resized.npy', X_train)
np.save('X_test_resized.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)


In [ ]:
# 다시 불러오기

# #X_train = np.load('X_train_resized.npy')
#X_test = np.load('X_test_resized.npy')
#y_train = np.load('y_train.npy')
#y_test = np.load('y_test.npy')


In [15]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [13]:
v_base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (244, 244, 3))

# 위에 새로운 FC 레이어 추가
x = Flatten()(v_base_model.output)
x = Dense(512, activation = 'relu')(x)
output = Dense(10, activation = 'softmax')(x)

v_model = Model(inputs = v_base_model.input, outputs = output)

# 사전 학습된 층은 동결( Fine-tuning의 경우 해제 가능)
# for layer in v_base_model.layers:
#    layer.trainable = False

# 모델 컴파일
v_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

v_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 244, 244, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 244, 244, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 244, 244, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 122, 122, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 122, 122, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 122, 122, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 61, 61, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 61, 61, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 61, 61, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 61, 61, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 30, 30, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 30, 30, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 30, 30, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,565,386 (105.15 MB)

 Trainable params: 27,565,386 (105.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
v_model.fit(X_train, y_train, epochs = 5, batch_size = 64, validation_data = (X_test, y_test))

Epoch 1/5
149/294 ━━━━━━━━━━━━━━━━━━━━ 19:04 8s/step - accuracy: 0.1053 - loss: 2.7312

In [ ]:
# 모델 평가
v_test_loss, v_test_acc = v_model.evalutate(X_test, y_test)
print(f"Test accuracy: {v_test_acc}")

NameError: name 'v_model' is not defined

In [24]:
#Use the generated model 
from keras.models import Model


output_vgg16_conv = model_vgg16_conv(vgg16_input)

#Add the fully-connected layers 

x = Flatten(name='flatten')(output_vgg16_conv)
# x = Dense(4096, activation='relu', name='fc1')(x)
# x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

vgg16_pretrained = Model(inputs = vgg16_input, outputs = x)
vgg16_pretrained.summary()

# Compile CNN model
sgd = optimizers.SGD(learning_rate = 0.001)
vgg16_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Image_input (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 predictions (Dense)         (None, 10)                250890    
                                                                 
Total params: 14965578 (57.09 MB)
Trainable params: 14965578 (57.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Running VGG-16 with Image Aumentation

C:\Users\nays0\AppData\Local\Temp\ipykernel_8092\124095390.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg16_model = vgg16_pretrained.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[checkpointer, earlystopper],


Epoch 1/25



InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/remove_squeezable_dimensions/Squeeze defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\nays0\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\nays0\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\nays0\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\nays0\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\nays0\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\nays0\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\nays0\anaconda3\Lib\asyncio\events.py", line 80, in _r